In [57]:
import numpy as np
import pandas as pd
import re
import glob
import os 
from sklearn.model_selection import GridSearchCV,cross_val_score,cross_val_predict
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from xgboost import XGBClassifier

In [2]:
path_train="../window/new_scores/new_stack_train/"
path_test="../window/new_scores/new_stack_test/"

In [3]:
dict_train = {} 
dict_test = {}
train_files = [i for i in os.listdir(path_train)]
test_files = [i for i in os.listdir(path_test)]
for f in train_files:
    dict_train[re.sub("\.csv","",f)] = pd.read_csv(path_train+f)
for f in test_files:
    dict_test[re.sub("\.csv","",f)] = pd.read_csv(path_test+f)

In [16]:
stack_train=pd.concat((dict_train["new_stem_xgb_bio"],dict_train["new_stem_xgb_f_tfidf"],
dict_train["new_stem_xgb_w_tfidf"],dict_train["new_stem_xgb_w2v"],dict_train["new_stem_lgbm_bio"],
dict_train["new_stem_lgbm_f_tfidf"],dict_train["new_stem_lgbm_w_tfidf"],dict_train["new_stem_lgbm_w2v"],
dict_train["new_stem_lr_bio"],dict_train["new_stem_lr_f_tfidf"],dict_train["new_stem_lr_w_tfidf"],
dict_train["new_stem_lr_w2v"],dict_train["new_stem_rf_bio"],dict_train["new_stem_rf_f_tfidf"],
dict_train["new_stem_rf_w_tfidf"],dict_train["new_stem_rf_w2v"]),axis=1)


In [19]:
stack_test=pd.concat((dict_test["new_stem_xgb_bio"],dict_test["new_stem_xgb_f_tfidf"],
dict_test["new_stem_xgb_w_tfidf"],dict_test["new_stem_xgb_w2v"],dict_test["new_stem_lgbm_bio"],
dict_test["new_stem_lgbm_f_tfidf"],dict_test["new_stem_lgbm_w_tfidf"],dict_test["new_stem_lgbm_w2v"],
dict_test["new_stem_lr_bio"],dict_test["new_stem_lr_f_tfidf"],dict_test["new_stem_lr_w_tfidf"],
dict_test["new_stem_lr_w2v"],dict_test["new_stem_rf_bio"],dict_test["new_stem_rf_f_tfidf"],
dict_test["new_stem_rf_w_tfidf"],dict_test["new_stem_rf_w2v"]),axis=1)

In [20]:
#quand le merging est fini : table unique, on peut monter un meta_model

In [54]:
new_train=pd.read_csv("../window/checkpoints_databases/new_working_train.csv")
y=np.array(new_train.Class)-1

In [75]:
ID_test=stack_test["ID"]
ID_test=ID_test.iloc[:,0]
ID_train=stack_train["ID"]
ID_train=ID_train.iloc[:,0]

In [23]:
feat_train=pd.read_csv("../window/w_meta_features/meta_train_l1l2.csv")
feat_test=pd.read_csv("../window/w_meta_features/meta_test_l1l2.csv")
feat_train=feat_train.drop("ID",axis=1)
feat_test=feat_test.drop("ID",axis=1)
feat_ext_train=pd.read_csv("../window/w_meta_features/new_working_train_ext.csv")
feat_ext_test=pd.read_csv("../window/w_meta_features/new_working_test_ext.csv")

In [24]:
stack_train_1=pd.concat((stack_train,feat_train,feat_ext_train),axis=1)
stack_test_1=pd.concat((stack_test,feat_test,feat_ext_test),axis=1)

In [46]:
X_train=np.array(stack_train_1.drop("ID",axis=1))

In [47]:
X_test=np.array(stack_test_1.drop("ID",axis=1))

In [48]:
kf = StratifiedKFold(n_splits=5, random_state=26, shuffle=True)

In [58]:
#Ridge vs Lasso searching
param_test= {
    "C" : [0.5,1,2,3,4,5],
    "penalty": ["l1","l2"]
}


In [59]:
gsearch=GridSearchCV(estimator=LogisticRegression(), param_grid = param_test,scoring="neg_log_loss",n_jobs=4,iid=False, cv=kf)
gsearch.fit(X_train,y)
gsearch.grid_scores_, gsearch.best_params_, gsearch.best_score_


C:\Users\Franck\Documents\Continuum\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:667: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


([mean: -0.91968, std: 0.03681, params: {'C': 0.5, 'penalty': 'l1'},
  mean: -0.92156, std: 0.03941, params: {'C': 0.5, 'penalty': 'l2'},
  mean: -0.92193, std: 0.03882, params: {'C': 1, 'penalty': 'l1'},
  mean: -0.92822, std: 0.04362, params: {'C': 1, 'penalty': 'l2'},
  mean: -0.93851, std: 0.04563, params: {'C': 2, 'penalty': 'l1'},
  mean: -0.94071, std: 0.04926, params: {'C': 2, 'penalty': 'l2'},
  mean: -0.96018, std: 0.05344, params: {'C': 3, 'penalty': 'l1'},
  mean: -0.95109, std: 0.05344, params: {'C': 3, 'penalty': 'l2'},
  mean: -0.97934, std: 0.05886, params: {'C': 4, 'penalty': 'l1'},
  mean: -0.95993, std: 0.05688, params: {'C': 4, 'penalty': 'l2'},
  mean: -0.99686, std: 0.06335, params: {'C': 5, 'penalty': 'l1'},
  mean: -0.96762, std: 0.05979, params: {'C': 5, 'penalty': 'l2'}],
 {'C': 0.5, 'penalty': 'l1'},
 -0.9196754350868126)

In [62]:
import winsound

winsound.PlaySound('sound.wav', winsound.SND_FILENAME)

In [28]:
#model with parameters found on gridsearch

In [65]:
def stacker_gen(X,X_test,y,classifier,name):
    kf = StratifiedKFold(n_splits=5, random_state=26, shuffle=True)
    fold = 0
    y_test=0
    for train_index, test_index in kf.split(X, y):
        
        fold += 1

        X_train, X_valid    = X[train_index],   X[test_index]
        y_train, y_valid    = y[train_index],   y[test_index]

        print("Fold", fold, X_train.shape, X_valid.shape)

        clf=classifier
        clf.fit(X_train,y_train)
        p_test = clf.predict_proba(X_test)
        y_test += p_test/5
    classes = "class1,class2,class3,class4,class5,class6,class7,class8,class9".split(',')
    subm = pd.DataFrame(y_test, columns=classes)
    subm['ID'] = ID_test
    
    subm.to_csv("../final_scores/new_stacked_test_{}.csv".format(name),index=False)
    

In [33]:
#parametres à rentrer

In [81]:
clf = LogisticRegression(C=0.5,penalty="l1")
stacker_gen(X_train,X_test,y,clf,"linear")

Fold 1 (2337, 194) (588, 194)
Fold 2 (2338, 194) (587, 194)
Fold 3 (2340, 194) (585, 194)
Fold 4 (2342, 194) (583, 194)
Fold 5 (2343, 194) (582, 194)


In [1]:
#Compare it with a Xgboost stacking if time

In [78]:
clf_xgb=XGBClassifier(max_depth=3, objective="multi:softprob",seed=26)
param_test= {
    "n_estimators": [100,200,300],
    "max_depth" : [2],
    "min_child_weight" : [2,3],
    "subsample":[1],
    "colsample_bytree":[0.85]
}

In [79]:
gsearch_xgb=GridSearchCV(estimator=clf_xgb, param_grid = param_test,scoring="neg_log_loss",n_jobs=-1,iid=False, cv=kf)
gsearch_xgb.fit(X_train,y)
gsearch_xgb.grid_scores_, gsearch_xgb.best_params_, gsearch_xgb.best_score_

C:\Users\Franck\Documents\Continuum\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:667: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


([mean: -0.87006, std: 0.03638, params: {'colsample_bytree': 0.85, 'max_depth': 2, 'min_child_weight': 2, 'n_estimators': 100, 'subsample': 1},
  mean: -0.90382, std: 0.04139, params: {'colsample_bytree': 0.85, 'max_depth': 2, 'min_child_weight': 2, 'n_estimators': 200, 'subsample': 1},
  mean: -0.93615, std: 0.04825, params: {'colsample_bytree': 0.85, 'max_depth': 2, 'min_child_weight': 2, 'n_estimators': 300, 'subsample': 1},
  mean: -0.86968, std: 0.03722, params: {'colsample_bytree': 0.85, 'max_depth': 2, 'min_child_weight': 3, 'n_estimators': 100, 'subsample': 1},
  mean: -0.90244, std: 0.04143, params: {'colsample_bytree': 0.85, 'max_depth': 2, 'min_child_weight': 3, 'n_estimators': 200, 'subsample': 1},
  mean: -0.93471, std: 0.04529, params: {'colsample_bytree': 0.85, 'max_depth': 2, 'min_child_weight': 3, 'n_estimators': 300, 'subsample': 1}],
 {'colsample_bytree': 0.85,
  'max_depth': 2,
  'min_child_weight': 3,
  'n_estimators': 100,
  'subsample': 1},
 -0.86968086988476134)

In [80]:
clf_xgb = XGBClassifier(n_estimators=100,max_depth=2,min_child_weight=3,subsample=1,colsample_bytree=0.85,seed=26,objective="multi:softprob")
stacker_gen(X_train,X_test,y,clf_xgb,"xgb")

Fold 1 (2337, 194) (588, 194)
Fold 2 (2338, 194) (587, 194)
Fold 3 (2340, 194) (585, 194)
Fold 4 (2342, 194) (583, 194)
Fold 5 (2343, 194) (582, 194)


In [84]:
pd.read_csv("../stage1_solution_filtered.csv").

(368, 10)